This document explains the "Explicit" (non-implicit) Treap datastructure, hereon referred to as just a "Treap". A solid understanding of Trees and Heaps is required to understand the inner workings of any Treap variant, and those concepts are partially covered in this explanation.

Written by Brandon

In [47]:
##Treap Quick Code Here
import random

class Treap:
    left = right = None
    def __init__(self, val):
        self.val = val
        self.weight = random.randint(1, int(1e9))

def split(node, x):
    if not node:
        return (None, None)
    if node.val <= x:
        left, right = split(node.right, x)
        node.right = left
        return (node, right)
    else:
        left, right = split(node.left, x)
        node.left = right
        return (left, node)
    
def merge(left, right):
    if not left or not right:
        return left if left else right
    if left.weight<=right.weight:
        left.right = merge(left.right, right)
        return left
    else:
        right.left = merge(left, right.left)
        return right
    
def add(root, val):
    node = Treap(val)
    left, right = split(root, val)
    right = merge(node, right)
    return merge(left, right)

def remove(root, val):
    left, right = split(root, val)
    left, _ = split(left, val-1)
    return merge(left, right)

def print_treap(root):
    l = print_treap(root.left) if root.left else []
    r = print_treap(root.right) if root.right else []
    return l + [root.val] + r

## Complexity

* Space Complexity: O(n) to create, O(1) for all other operations
* Creation: O(nlogn)
  * Can be modified to be  O(n) if initialized from a sorted sequence
* Insertion: O(logn)
* Removal: O(logn)
* Merge: O(log(n*m))
  * Where m and n are the size of each treap respectively

## What is a Treap

A Treap is exactly what it sounds like; a Tree and a Heap in one. It uses randomness to balance a Binary Search Tree. One could think of it this way: A non-self-balancing tree remains mostly balanced with a random stream of inputs, until corner cases cause its height to approach O(n) rather than O(logn) (e.g. adding only ascending values). Pretty much any useful operation is O(H), where H is the height of the tree, so it's important to make sure the tree is Height Balanced, (i.e. H = O(logn)). It's deceptively tricky to come up with an algorithm that does this reliably; Red-Black Trees and B-Trees are inconvenient to build, maintain and debug.

But what if there were a way to internally enforce a random structure, effectively ensuring that none of those corner-cases arise? Then the tree would be effectively bounded to an O(logn) height without any crazy self-balancing algorithms. While an imbalance is still theoretically possible, it is vanishingly unlikely. That's effectively the intuitin behind a Treap.

The idea is that every element is paired with a random priority. The BST invariant (that an inorder traversal yields ascending values) is maintained for the elements. The heap invariant (that each node has two children with lower priorities than the current node) is partially maintained, in that it's maintained on a local scale, which has the overarching effect of producing a height-balanced tree, when the priorities are random.

The two methods that Treaps use to modify their structure are Split(), and Merge():

Split(x) splits a Treap into two Treaps; one with values < x, and one with values >= x (or <= and >), and returns both of these new Treaps. This can be thought of like cutting down through the tree, with nodes falling on either side of the cut, and resultingly being included in the left or right Treaps that are created by the cut.

Merge(l, r) is the opposite of split; two Treaps are combined, and the resulting Treap is returned. This is where the Heap property is maintained. Since we know that l and r are both Treaps, they are both height-balanced. One of these Treaps' roots must be the root of the combined Treap, and the choice of which to elect as the new root to keep the structure height-balanced is called 'height control'. Here, height control just comes down to selecting the root randomly, according to the priority values of l and r, as if to maintain the Heap invariant. Because this is based on internally randomized variables, and not anything external (like order of input, or always selecting one over the other), this ensures (statistically) an O(logn) height for the new Heap

The main utility of a Treap is that it's a self-balancing binary tree. This corresponds to the built-in std::map in C++, the sortedcontainers in Python (namely SortedDict), or the TreeMap in Java. If you're looking for a simple way to store elements, such that they have both O(logn) lookup and indexing, use one of those. Explicit treaps have very specific legitimate use-cases (mentioned below), but for many situations a built-in ordered map achieves the same result.

There are many self-balancing binary search trees, most notably the Red Black Tree. AVL Trees are probably the second most prominent, with B Trees and Treaps trailing distantly behind that. The "self-balancing" quality of a Binary Search Tree is extremely valuable, because it ensures the height of the BST doesn't exceed O(logn). This means that any path from the root to another node is at most O(logn), which ensures O(logn) indexing and lookup. Otherwise, for a *non*-height-balanced tree, a case can arise where elements are inserted in an order such that the tree has an O(n) height, with each node only having one child (e.g. resembling a linked list). The operational complexity for a non-balanced BST is resultingly bounded to O(n), which defeats the purpose of the structure.

## Why Use a Treap?

* Almost always, a Treap refers to an Implicit Treap. There are several small reasons to use a basic Treap over a comparable data structure, listed below, but the use-case for a basic Treap is miniscule when almost every language has a built-in map structure. The most important thing is just to understand the structure of a basic Treap, as a stepping stone towards understanding the structure of an Implicit Treap.

* The immediate question to ask is, "Should I just use a built-in map instead?" If the intent is just to have an ordered datastructure, then the answer is yes: use a built-in map instead. A reason one would use a Treap, or any other structure as opposed to a built-in map equivalent, is because it is modifiable.

* The biggest reason one would use a Treap over another handwritten self-balancing tree is because of its simplicity. Red Black Trees are a nightmare to implement, with many, many lines of code. In a timed setting a treap is trivial in comparison to any other comparable self-balanacing tree, if you really needed to hand-write a self-balancing BST for some reason.

* This simplicity extends not only to the brevity of code, but also conceptual clarity. It's relatively easy to understand the mechanics of a Treap.

* Speed is another factor. While their worst-case times are abysmal (Linear for every operation), in practice Treaps have less overhead than other comparable structures, because the logic is so simple.

* A less apparent, but useful feature is synchronization. In concurrent systems, where multiple processes are accessing and possibly modifying a given data-structure at once, synchronization needs to be ensured. This means that, if multiple processes are observing or changing a given structure, they are able to do so concurrently with no issue. Due to the random and modular nature of Treaps, they are much more condusive to parallel processing than other data structures. Red Black Trees do not have this quality, with the Java TreeMap documentation explicitly requiring external synchronization.

* The structure of a Treap is based on the (random) priority generation, and not order of insertion. This means that, if a deterministic method is used to generate priorities (such as a hash function of the element), then the structure of every such Treap with the same elements will have the exact same structure, regardless of order of insertion.

* Another minute result of this is that an adversary could not deduce the order of insertion from a Treap, where in theory they could from another form of self-balancing tree.

## Building a Treap:

A Treap is organzed in terms of nodes, so the first step is to define the node class. This tutorial will define the Treap functionality in terms of this object, and associated methods. This coult be formatted any number of ways.

In [1]:
import random
class Treap:
    left = right = None
    def __init__(self, val):
        self.val = val
        self.weight = random.randint(1, int(1e9))

We can see above that the Treap node contains pointers to children (which are initially null), a given value, and a random priority.

The main two methods of a Treap are Split, and Merge, as mentioned above. Let's start with Split. The idea is that we can build a recursive function, that takes a Treap and a value x, and returns two Treaps; one of everything less than the value, and one of everything greater than the value.

In [2]:
def split(node, x):
    if not node:
        return (None, None)
    if node.val <= x:
        left, right = split(node.right, x)
        node.right = left
        return (node, right)
    else:
        left, right = split(node.left, x)
        node.left = right
        return (left, node)

This split method operates recutsively, at every layer of the Treap. Let's go through the logic here.

The first thing is to check that the recursive call hasn't reached the bottom of the Treap (e.g. when node is null). In that case, just return nothing, because there are no more nodes to attach to anything.

The main choice to make at each step is, which sub-Treap to include node in. This just comes down to whether node.val <= x (in which case it goes in the left returned Treap, or else it goes in the right). If node.val <= x, then we need to examine node.right, to see which subtrees of node.right are on either side of x. In this case, all nodes in node.left are <= x, so there's no need to check them; they can all be safely added to the left Treap. If node.val > x, then we know that all values in node.right are also > x, so they don't need to be checked; they will all be included in right.

One important thing to recognize is that this function only executes on nodes closest to x. This means the recursive path draws a line down the Treap, following x, which is O(H) recursive calls. Because we can assume the Treap is height-controlled, this is O(logn) recursive calls.

Next is the merge method, which pretty much does the exact inverse of the split method. It takes two Treaps and combines them into one. This is where the concept of a priority, or node.weight comes in. If we only looked at the values, then there is no height control in either function, and this would not be a height balanced structure.

In fact, merge makes the assumption that left is entirely less than right, in that all nodes in left have lower values than right. This naturally happens if both Treaps were created previously by Split(). Becuase of this assumption, just as Split made no reference to weights, Merge makes no references to values.

In [3]:
def merge(left, right):
    if not left or not right:
        return left if left else right
    if left.weight<=right.weight:
        left.right = merge(left.right, right)
        return left
    else:
        right.left = merge(left, right.left)
        return right

If one Treap is empty, then a Treap merged with nothing is just that same Treap.

After the base case is covered, the central decision in the function is whether to elect left to be the root, with right being in left's subtree, or to elect right to be the root. This comes down to which node has a higher weight, maintaining our max-heap property, at least locally. This is where the height is all but ensured to be O(logn).

With these two methods, we have the logic to create other more typical functions.

In [39]:
def add(root, val):
    node = Treap(val)
    left, right = split(root, val)
    right = merge(node, right)
    return merge(left, right)

def remove(root, val):
    left, right = split(root, val)
    left, _ = split(left, val-1)
    return merge(left, right)

def print_treap(root):
    l = print_treap(root.left) if root.left else []
    r = print_treap(root.right) if root.right else []
    return l + [root.val] + r

In [46]:
root = Treap(5)
for _ in range(10):
    root = add(root, random.randint(1, 10))
print_treap(root)

[2, 2, 3, 4, 5, 6, 6, 9, 10, 10, 10]

That's a completed Treap! No matter what values are added to it, in what order, a (statistically) height-balanced BST will result.

## Notes:

The Treap's worst-case time complexity is actually linear (rather than logn), though the chance of this is effectively 0. Unlike in a non-balanced tree, this is not affected in any way by the structure/order of the input, making it reliable where a non-balanced tree isn't.

One might wonder; what self-balancing trees are used in the built-in classes for each language?

C++ and Java implement Red Black Trees in their map implementations, perhaps giving the structure its notoriety. This is because Red Black Trees are more reliable than Treaps, and faster (in a general use-case) than AVL trees.
Python's sortedcontainers actually use multidimensional lists, as opposed to any tree structure, for their map implementations. This utilizes built-in hardware optimizations to surpass the speed of other O(logn) implementations (such as self-balancing trees), for all conceivable benchmarks.

A Treap is also called a Cartesian Tree, because each element has two dimensions (the value, and the priority). It is often depicted as a two-dimensional tree on the Cartesian plane.

An Implicit Treap is a separate data-structure based on the Treap, where the priority of an element is determined by how many nodes are less than it (when this is implicit, O(n) priorities can be changed in O(1) complexity). This enables operations similar to Segment Trees and Fenwick Trees, such as range queries, and range updates. As mentioned above, a basic Treap has little use-case, but an Implicit Treap is much more useful, especially for how simple it is.

## Sources:

https://en.wikipedia.org/wiki/Treap

https://en.wikipedia.org/wiki/Implicit_data_structure

https://en.wikipedia.org/wiki/Self-balancing_binary_search_tree

https://en.wikipedia.org/wiki/Red%E2%80%93black_tree

https://en.wikipedia.org/wiki/B-tree

https://en.cppreference.com/w/cpp/container/map

https://grantjenks.com/docs/sortedcontainers/implementation.html

https://docs.oracle.com/javase/8/docs/api/java/util/TreeMap.html

https://stackoverflow.com/questions/5288320/why-is-stdmap-implemented-as-a-red-black-tree

https://stackoverflow.com/questions/16009361/when-to-use-a-treap

https://www.cs.cmu.edu/~scandal/papers/treaps-spaa98.pdf

https://cp-algorithms.com/data_structures/treap.html#toc-tgt-1

https://usaco.guide/adv/treaps?lang=cpp#implicit-treaps

